<span style="font-size: 14pt">MIPT, Advanced ML, Autumn 2017</span>

<span style="font-size: 16pt"> HW #9: Recommendation System

<span style="color:blue; font-size: 12pt">Andrey Saitgalin </span>
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">andrey.saitgalin@gmail.com</span>, 

<span style="color:blue; font-size: 12pt">Alexey Dral </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">aadral@gmail.com</span>

<h1 align="center">Organization Info</h1> 

**Дополнительный материал для выполнения дз**:
<a href= "https://github.com/ml-mipt/ml-mipt-part2/tree/master/2017/lectures">Лекции по курсу</a>

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall <номер_группы> <фамилия>``, к примеру -- ``ML2017_Aut_fall 401 ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb``, к примеру -- ``ivanov_491_task9.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall Question <Содержание вопроса>``

** Дедлайн**:
<span style="color:red; font-size: 12pt">16.12.2017 23:59 </span>

--------
- **PS1**: Мы используем автоматические фильтры, и просто не найдем ваше дз, если вы не аккуратно его подпишите.
- **PS2**: Напоминаем, что дедлайны жесткие, письма пришедшие после автоматически удаляются =( чтобы соблазна не было.

<h1 align="center">Check Questions (0%)</h1> 
Вопросы отсутствуют

<h1 align="center"> Practical tasks (100%)</h1>


# Построение content-based рекомендательной системы
## Part 1
Для каждого фильма получите описание в текстовом виде с IMDB или (и) The movie database (у них есть открытое API). https://www.themoviedb.org/?language=ru. Вывести описание фильма под id 778. Также можно использовать рецензии.

In [23]:
import tmdbsimple as tmdb
from tqdm import trange
tmdb.API_KEY = "82d15a009321e054584df26bf7530dc7"

movies = {}

for i in trange(1000):
    try:
        movies[i] = tmdb.Movies(i).info()['overview']
    except Exception:
        continue

100%|██████████| 1000/1000 [19:55<00:00,  1.20s/it]


In [70]:
movies[99116] = tmdb.Movies(99116).info()['overview']

In [24]:
movies[778]

'Monsieur Hulot comes to a beachside hotel for a vacation, where he accidentally (but good-naturedly) causes havoc.'

## Part 2 
Извлеките текстовые признаки у фильмов: tf idf, биграммы (может какие-то еще). Биграммы - статистика по биграммам, сколько какая биграмма встречается. Формулы для TF-IDF смотреть тут: https://en.wikipedia.org/wiki/Tf%E2%80%93idf 
    1. Используйте TF с double 0.5 normalization
    2. Используйте стандартный IDF: 
$$idf(t, D) = \frac{log N}{(1 + |d in D : t in d|)}$$

In [207]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(ngram_range=(1, 2)) 
count_vectorizer_1 = CountVectorizer(ngram_range=(1, 1))

In [71]:
sparse_feature_matrix = count_vectorizer.fit_transform(list(movies.values()))
sparse_feature_matrix

<825x36683 sparse matrix of type '<class 'numpy.int64'>'
	with 71403 stored elements in Compressed Sparse Row format>

In [209]:
sparse_feature_matrix_1 = count_vectorizer_1.fit_transform(list(movies.values()))
sparse_feature_matrix_1

<825x8230 sparse matrix of type '<class 'numpy.int64'>'
	with 32867 stored elements in Compressed Sparse Row format>

In [73]:
index_to_id = np.unique(np.sort(list(movies.keys())))
id_to_index = {}

for index, value in enumerate(index_to_id):
    id_to_index[int(value)] = index

assert len(index_to_id) == len(movies)

In [210]:
def tf_idf(t, d, D):
    
    tf = 0.5 + 0.5*(sparse_feature_matrix_1[id_to_index[d], t]/
                    max(sparse_feature_matrix_1[id_to_index[d], :].data))
    idf = np.log(len(D)/len(sparse_feature_matrix_1[:, t].indices))
    
    return tf * idf

In [211]:
data_tf_idf = []
row_ind = []
col_ind = []
for d in index_to_id:
    for t in sparse_feature_matrix_1[id_to_index[d], :].indices:
        data_tf_idf.append(tf_idf(t, d, index_to_id))
        row_ind.append(id_to_index[d])
        col_ind.append(t)

In [212]:
from scipy.sparse import csr_matrix

tf_idf_sparse_matrix = csr_matrix((np.array(data_tf_idf), (np.array(row_ind), np.array(col_ind))),
                                  shape=np.shape(sparse_feature_matrix))

## Part 3 
Выведите подсчитанные признаки из пункта 2 для фильма с id 99114.

In [219]:
num_2_words_bigrams = {
    v: k
    for k, v in count_vectorizer.vocabulary_.items()
}
num_2_words = {
    v: k
    for k, v in count_vectorizer_1.vocabulary_.items()
}

In [222]:
# There is no film with id 99114 on tmdb, so i have chosen the nearest one (id 99116)
for i in sparse_feature_matrix[id_to_index[99116], :].indices:
    print(num_2_words_bigrams[i], sparse_feature_matrix[id_to_index[99116], i])

released in 1
and released 1
vaallidharu and 1
as vaallidharu 1
telugu as 1
in telugu 1
rangeela and 1
mr rangeela 1
as mr 1
hindi as 1
into hindi 1
dubbed into 1
later dubbed 1
was later 1
superhit at 1
declared superhit 1
was declared 1
2002 and 1
november 2002 1
29 november 1
on 29 1
released on 1
panneerselvam released 1
by panneerselvam 1
handled by 1
cinematography handled 1
and cinematography 1
raja and 1
shankar raja 1
yuvan shankar 1
by yuvan 1
scored by 1
music scored 1
had music 1
which had 1
film which 1
cast the 1
supporting cast 1
the supporting 1
karunas among 1
and karunas 1
devan and 1
prabhu devan 1
venkat prabhu 1
jayaraman venkat 1
gayatri jayaraman 1
with gayatri 1
roles with 1
lead roles 1
the lead 1
sneha in 1
and sneha 1
srikanth and 1
starring srikanth 1
stanley starring 1
newcomer stanley 1
by newcomer 1
and directed 1
written and 1
film written 1
romantic drama 1
tamil romantic 1
indian tamil 1
2002 indian 1
is 2002 1
april is 1
of april 1
month of 1
the mont

In [223]:
for i in tf_idf_sparse_matrix[id_to_index[99116], :].indices:
    print(num_2_words[i], tf_idf_sparse_matrix[id_to_index[99116], i])

2002 3.87143756086
2004 3.20958348438
29 3.83736193505
among 2.64910962552
and 0.258613730763
april 3.61078141993
as 0.89509719034
at 1.15659171663
box 3.04519372869
by 0.953918594853
cast 2.52159902477
cinematography 3.83736193505
declared 3.44127783187
devan 3.83736193505
directed 2.8134993812
drama 2.15482537552
dubbed 3.04519372869
english 2.41741527803
film 1.2369834033
gayatri 3.83736193505
had 2.18572093054
handled 3.83736193505
hindi 3.44127783187
in 0.429235961068
indian 2.8134993812
into 1.07377227393
is 0.406569325273
jayaraman 3.83736193505
karunas 3.83736193505
later 2.28990467728
lead 2.37167658793
maadhathil 3.83736193505
month 3.44127783187
mr 2.72541327845
music 2.52159902477
newcomer 3.44127783187
november 3.44127783187
of 0.168793404679
office 2.8134993812
on 0.693785814737
others 2.72541327845
panneerselvam 3.83736193505
prabhu 3.83736193505
raja 3.83736193505
rangeela 3.83736193505
released 3.28239351894
roles 3.20958348438
romantic 2.52159902477
scored 3.837361935

## Part 4
Для каждого пользователя найдите топ жанров, которые нравятся пользователю (можно делать как на семинаре top 5, top 3 и top 2) и порекомендуйте таким пользователя фильмы близкие по текстовому описанию (см. пункт 2). Для оценки результатов пункта 3 используйте метрику HitRate и метрики ранжирования (N)DCG. Формулу для подсчета HitRate можно взять на семинаре,  (N)DCG: https://en.wikipedia.org/wiki/Discounted_cumulative_gain



In [273]:
tmdb.API_KEY = '82d15a009321e054584df26bf7530dc7'
users = {}

for i in trange(10):
    try:
        users[i] = tmdb.Account(i)
    except Exception:
        continue

100%|██████████| 10/10 [00:00<00:00, 29641.72it/s]


In [274]:
users[0].info()

HTTPError: 401 Client Error: Unauthorized for url: https://api.themoviedb.org/3/account?session_id=0&api_key=82d15a009321e054584df26bf7530dc7

In [257]:
import http.client

conn = http.client.HTTPSConnection("api.themoviedb.org")

payload = "{}"

conn.request("GET", "/3/account/%7Baccount_id%7D/favorite/movies?page=1&sort_by=created_at.asc&language=en-US&api_key=6610a2df2a35548473ddd5aac84404b1", payload)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"status_code":3,"status_message":"Authentication failed: You do not have permissions to access the service."}
